In [1]:
import cvxpy as cp
import numpy as np

In [155]:
def DCOPF(Y,PGl,PGu,PD,thetaL,thetaU,CQ,CL,PF,slack):
    num_buses = len(Y)
    num_generators = PGl.shape[1]
    B = np.imag(Y)
    t = len(PD)

    PG = cp.Variable((t,num_generators)); # optimization variable: PG_i
    theta = cp.Variable((t,num_buses)); # optimization variable: theta_i

    objective = cp.Minimize(cp.sum(PG**2 * CQ.T + PG * CL.T)) # objective function

    constraints = [
        # Power generation constraints
        PG - PGu <= 0,
        -PG + PGl <= 0,

        #Bus phase angle constraints
        theta.T - thetaU <= 0,
        -theta.T + thetaL <= 0,

        #Power balance constraint
        PG == theta * B + PD,

        #Slack angle constraint   
        theta[:,slack] == 0
    ]
    
    # Line capacity constraints
    count = 0
    for i in range(num_buses):          
        for j in range(i+1, num_buses):
            constraints.append(B[i,j] * (theta[:,i] - theta[:,j]) - PF[:,count] <= 0)
            constraints.append(B[i,j] * (theta[:,j] - theta[:,i]) - PF[:,count] <= 0)
            count = count + 1

    # SOLVE IT
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    print(result)
    
    pf_opt = np.zeros((t,num_buses,num_buses))
    for i in range(num_buses):
        for j in range(i+1, num_buses):
            pf_opt[:,i,j] = B[i,j] * (theta[:,i].value - theta[:,j].value)
    
    cost = objective.value
    p_opt = PG.value
    theta_opt = theta.value
    lmp_opt = constraints[4].dual_value.reshape((t,num_buses))
    
    return pf_opt, cost, p_opt, theta_opt, lmp_opt

In [157]:
NUM_HOURS = 31*24
NUM_BUSES = 3
Y = np.array([[1/0.5j+1/0.2j, -1/0.5j, -1/0.2j],
    [-1/0.5j, 1/0.5j+1/0.4j, -1/0.4j],
    [-1/0.2j, -1/0.4j, 1/0.2j+1/0.4j]])

PGl = np.array([[0, 0, 0]]).repeat(NUM_HOURS, axis=0)
PGu = np.array([[1, 1, 0]]).repeat(NUM_HOURS, axis=0)
thetal = -np.ones((3,NUM_HOURS))
thetau = np.ones((3,NUM_HOURS))
CQ = np.array([[1, 1, 0]])
CL = np.array([[1, 1, 0]])
PF = np.array([[1, 1, 1]]).repeat(NUM_HOURS, axis=0)
slack= 1
PD = np.array([[0, 0.3, 0.4]]).repeat(NUM_HOURS, axis=0)

pf_opt, cost, p_opt, theta_opt, lmp_opt = DCOPF(Y, PGl, PGu, PD, thetal, thetau, CQ, CL, PF, slack)
print('OPF:', pf_opt)
print('Total cost:', cost)
print('Generation:', p_opt)
print('Angles:', theta_opt)
print('LMPs: ', lmp_opt)

703.0799999480035
OPF: [[[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]

 [[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]

 [[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]

 ...

 [[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]

 [[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]

 [[ 0.         -0.04545389 -0.30454491]
  [ 0.          0.         -0.09545509]
  [ 0.          0.          0.        ]]]
Total cost: 703.0800000019616
Generation: [[3.49998797e-01 3.50001203e-01 2.04595196e-11]
 [3.49998797e-01 3.50001203e-01 2.04595196e-11]
 [3.49998797e-01 3.50001203e-01 2.04595196e-11]
 ...
 [3.49998797e-01 3.50001203e-01 2.04595196e-11]
